# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [ ]:
!pip install catboost

In [3]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from scipy.stats import mode 
from catboost import CatBoostClassifier

In [4]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [5]:
class DecisionTreeLeaf:
    def __init__(self, y):
        self.y = mode(y)[0].squeeze()


class DecisionTreeNode:
    def __init__(self, split_dim, left, right):
        self.split_dim = split_dim
        self.left = left
        self.right = right

In [6]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.X, self.y = X, y
        self.train_indices = np.random.choice(len(X), size=len(X), replace=True)
        self.oob_indices = set(np.setdiff1d(np.arange(len(X)), self.train_indices))
        self.criterion = gini if criterion == "gini" else entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = round(np.sqrt(X.shape[1])) if max_features == "auto" else max_features
        self.root, self.depth = self._build_tree(self.train_indices, 0)
  
    def _build_tree(self, indices, depth):
        if (self.max_depth is not None) and (depth >= self.max_depth):
            return DecisionTreeLeaf(self.y[indices]), depth

        partition = self._get_partition(indices)

        if partition is None:
            return DecisionTreeLeaf(self.y[indices]), depth

        left_indices, right_indices, split_dim = partition
        left_tree, left_depth = self._build_tree(left_indices, depth + 1)
        right_tree, right_depth = self._build_tree(right_indices, depth + 1)
        return DecisionTreeNode(split_dim, left_tree, right_tree), max(left_depth, right_depth)

    def _get_partition(self, indices):
        X, y = self.X[indices], self.y[indices]
        max_gain = float('-inf')
        split_dim = left_indices = right_indices = None
        features = np.random.choice(X.shape[1], size=self.max_features, replace=False)

        for feature in features:
            left, right = indices[X[:, feature] == 0], indices[X[:, feature] == 1]

            if (len(left) < self.min_samples_leaf) or (len(right) < self.min_samples_leaf):
                continue

            gain_ = gain(self.y[left], self.y[right], self.criterion)
            
            if gain_ > max_gain:
                max_gain = gain_
                split_dim = feature
                left_indices, right_indices = left, right
        
        return None if split_dim is None else (left_indices, right_indices, split_dim)
        
    def predict(self, X):
        if X.ndim == 1:
            X = X.reshape(1, -1)

        predictions = []
        for x in X:
            node = self.root 

            while not isinstance(node, DecisionTreeLeaf):
                node = node.left if x[node.split_dim] == 0 else node.right
            predictions.append(node.y)

        return predictions

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [7]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.X = self.y = self.estimators = None
    
    def fit(self, X, y):
        self.X, self.y = X, y
        self.estimators = []
        
        for _ in range(self.n_estimators):
            tree = DecisionTree(X, y, self.criterion, self.max_depth, 
                                self.min_samples_leaf, self.max_features)
            self.estimators.append(tree)
    
    def predict(self, X):
        predictions = self.estimators[0].predict(X)

        for estimator in self.estimators[1:]:
            predictions = np.vstack((predictions, estimator.predict(X)))
        
        return mode(predictions, axis=0)[0].squeeze()

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [8]:
def oob_error(rfc, X, y):
    true = size = 0

    for index, x in enumerate(X):
        pred = [estimator.predict(x)[0] for estimator in rfc.estimators 
                if index in estimator.oob_indices]

        if len(pred) > 0:
            size += 1
            pred = mode(pred)[0][0]
            true += (pred == y[index])

    return 1 - true / size


def feature_importance(rfc):
    importance = []
    err_oob = oob_error(rfc, rfc.X, rfc.y)

    for i in range(rfc.X.shape[1]):
        X_new = rfc.X.copy()
        np.random.shuffle(X_new[:, i])
        err_oob_i = oob_error(rfc, X_new, rfc.y)
        importance.append(err_oob_i - err_oob)

    return importance


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    indices = np.argsort(importance)[::-1][:k]
    return np.array(names)[indices]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [9]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [0.0, 0.0, 0.19899999999999995, 0.20199999999999996, 0.45699999999999996, 0.0]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [10]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [11]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [12]:
%%time
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7490542244640606
Most important features:
1. ovsyanochan
2. mudakoff
3. 4ch
4. rhymes
5. styd.pozor
6. dayvinchik
7. ohhluul
8. rapnewrap
9. pravdashowtop
10. i_d_t
11. pixel_stickers
12. pozor
13. reflexia_our_feelings
14. bot_maxim
15. leprum
16. thesmolny
17. iwantyou
18. tumblr_vacuum
19. borsch
20. i_des
CPU times: user 4min 27s, sys: 368 ms, total: 4min 27s
Wall time: 4min 27s


#### Пол

In [13]:
%%time
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8524590163934426
Most important features:
1. 40kg
2. zerofat
3. mudakoff
4. modnailru
5. girlmeme
6. 9o_6o_9o
7. femalemem
8. recipes40kg
9. woman.blog
10. i_d_t
11. thesmolny
12. cook_good
13. be.beauty
14. igm
15. be.women
16. bon
17. 4ch
18. reflexia_our_feelings
19. beauty
20. soverwenstvo.decora
CPU times: user 4min 23s, sys: 262 ms, total: 4min 23s
Wall time: 4min 23s


### CatBoost
В качестве альтернативы попробуем CatBoost. 

Установить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [14]:
X, y = synthetic_dataset(1000)
model = CatBoostClassifier(iterations=10, 
                           learning_rate=1, 
                           depth=2, 
                           loss_function='MultiClass',
                           random_seed=0)
model.fit(X, y, verbose=False)
print("Accuracy:", np.mean(model.predict(X).squeeze() == y))
print("Importance:", model.get_feature_importance())

Accuracy: 1.0
Importance: [ 0.          0.         22.24583462 28.33697139 49.41719399  0.        ]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [15]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [16]:
%%time
model = CatBoostClassifier(iterations=100, loss_function='MultiClass', random_seed=0)
grid = {'learning_rate': [0.01, 0.1, 1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_search_result = model.grid_search(grid, 
                                       X=X_train, 
                                       y=y_age_train, 
                                       partition_random_seed=0,
                                       verbose=False)

CPU times: user 26min 44s, sys: 10.5 s, total: 26min 54s
Wall time: 13min 51s


In [17]:
grid_search_result['params']

{'depth': 10, 'l2_leaf_reg': 1, 'learning_rate': 0.1}

In [18]:
model = CatBoostClassifier(iterations=100, 
                           **grid_search_result['params'],
                           loss_function='MultiClass',
                           random_seed=0)
model.fit(X_train, y_age_train, verbose=False)
print("Accuracy:", np.mean(model.predict(X_test).squeeze() == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.725094577553594
Most important features:
1. ovsyanochan
2. mudakoff
3. styd.pozor
4. 4ch
5. rhymes
6. dayvinchik
7. rapnewrap
8. leprum
9. iwantyou
10. xfilm


#### Пол

In [ ]:
%%time
model = CatBoostClassifier(iterations=100, loss_function='MultiClass', random_seed=0)
grid = {'learning_rate': [0.01, 0.1, 1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_search_result = model.grid_search(grid, 
                                       X=X_train, 
                                       y=y_sex_train, 
                                       partition_random_seed=0,
                                       verbose=False)

In [20]:
model = CatBoostClassifier(iterations=100, 
                           **grid_search_result['params'],
                           loss_function='MultiClass',
                           random_seed=0)
model.fit(X_train, y_sex_train, verbose=False)
print("Accuracy:", np.mean(model.predict(X_test).squeeze() == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8474148802017655
Most important features:
1. 40kg
2. mudakoff
3. igm
4. girlmeme
5. zerofat
6. modnailru
7. 9o_6o_9o
8. academyofman
9. be.beauty
10. thesmolny
